In [14]:
import graphviz

def visualize_multihead_attention_with_aspect():
    # Create a new directed graph
    graph = graphviz.Digraph(comment='MultiheadAttentionWithAspect', graph_attr={'rankdir': 'TB', 'size': '12,12'})

    # Input nodes
    graph.node('sentence_embedding', 'Sentence Embedding\n[0.2, 0.5, ..., 0.1]\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('aspect_embedding', 'Aspect Embedding\n[0.1, 0.3, ..., 0.2]\nShape: (aspect_length, d_model)', shape='box', style='filled', fillcolor='lightblue')

    # Concatenation and linear transformation
    graph.node('concatenation', 'Concatenation\nCombines sentence and aspect embeddings\nShape: (batch_size, seq_length, 2 * d_model)', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('query_linear', 'Query Linear\nTransforms concatenated embeddings\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightyellow')
    graph.edge('sentence_embedding', 'concatenation', label='sentence_embedding')
    graph.edge('aspect_embedding', 'concatenation', label='aspect_embedding')
    graph.edge('concatenation', 'query_linear', label='concatenated_query')

    # Layer normalization and positional encoding
    graph.node('layer_norm', 'Layer Normalization\nNormalizes the query tensor\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightpink')
    graph.node('pos_encoding', 'Positional Encoding\nAdds positional information to the query\nShape: (seq_length, d_model)', shape='box', style='filled', fillcolor='lightyellow')
    graph.edge('query_linear', 'layer_norm', label='query')
    graph.edge('layer_norm', 'pos_encoding', label='normalized_query')

    # Linear transformations for Q, K, and V
    graph.node('wq', 'Linear (Q)\nCreates query vectors\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('wk', 'Linear (K)\nCreates key vectors\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('wv', 'Linear (V)\nCreates value vectors\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightblue')
    graph.edge('pos_encoding', 'wq', label='query')
    graph.edge('pos_encoding', 'wk', label='query')
    graph.edge('pos_encoding', 'wv', label='query')

    # Reshaping and permutation for multi-head attention
    graph.node('reshape_q', 'Reshape (Q)\nReshapes query for multi-head attention\nShape: (batch_size, num_heads, seq_length, depth)', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('reshape_k', 'Reshape (K)\nReshapes key for multi-head attention\nShape: (batch_size, num_heads, seq_length, depth)', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('reshape_v', 'Reshape (V)\nReshapes value for multi-head attention\nShape: (batch_size, num_heads, seq_length, depth)', shape='box', style='filled', fillcolor='lightgreen')
    graph.edge('wq', 'reshape_q', label='Q')
    graph.edge('wk', 'reshape_k', label='K')
    graph.edge('wv', 'reshape_v', label='V')

    # Multi-head attention
    graph.node('matmul_qk', 'Matrix Multiplication (Q, K)\nComputes attention scores\nShape: (batch_size, num_heads, seq_length, seq_length)', shape='box', style='filled', fillcolor='lightyellow')
    graph.node('scale', 'Scaling\nScales attention scores\nShape: (batch_size, num_heads, seq_length, seq_length)', shape='box', style='filled', fillcolor='lightpink')
    graph.node('softmax', 'Softmax\nNormalizes attention scores\nShape: (batch_size, num_heads, seq_length, seq_length)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('dropout', 'Dropout\nRegularizes attention scores\nShape: (batch_size, num_heads, seq_length, seq_length)', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('matmul_av', 'Matrix Multiplication (Attention, V)\nComputes attention-weighted values\nShape: (batch_size, num_heads, seq_length, depth)', shape='box', style='filled', fillcolor='lightyellow')
    graph.edge('reshape_q', 'matmul_qk', label='Q')
    graph.edge('reshape_k', 'matmul_qk', label='K')
    graph.edge('matmul_qk', 'scale', label='attention_scores')
    graph.edge('scale', 'softmax', label='scaled_attention_scores')
    graph.edge('softmax', 'dropout', label='attention_probs')
    graph.edge('dropout', 'matmul_av', label='attention')
    graph.edge('reshape_v', 'matmul_av', label='V')

    # Reshape and linear transformation
    graph.node('reshape_output', 'Reshape Output\nReshapes attention output\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('fc_out', 'Linear\nTransforms reshaped output\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightyellow')
    graph.edge('matmul_av', 'reshape_output', label='weighted_value')
    graph.edge('reshape_output', 'fc_out', label='reshaped_output')

    # Residual connection and layer normalization
    graph.node('residual', 'Residual Connection\nAdds query to the transformed output\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightpink')
    graph.node('layer_norm_output', 'Layer Normalization\nNormalizes the output\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightblue')
    graph.edge('fc_out', 'residual', label='output')
    graph.edge('pos_encoding', 'residual', label='query')
    graph.edge('residual', 'layer_norm_output', label='residual_output')

    # Output nodes
    graph.node('output', 'Output\nContextual Embeddings\n[0.7, 0.2, ..., 0.5]\nShape: (batch_size, seq_length, d_model)', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('attention_weights', 'Attention Weights\n[[0.1, 0.2, ..., 0.05], ...]\nShape: (batch_size, num_heads, seq_length, seq_length)', shape='box', style='filled', fillcolor='lightgreen')
    graph.edge('layer_norm_output', 'output', label='output')
    graph.edge('dropout', 'attention_weights', label='attention')

    # Render and save the diagram
    graph.render('multihead_attention_with_aspect_detailed', view=True)

# Call the function to generate the diagram
visualize_multihead_attention_with_aspect()

In [18]:
import graphviz

def visualize_multihead_attention_with_aspect():
    # Create a new directed graph
    graph = graphviz.Digraph(comment='MultiheadAttentionWithAspect', graph_attr={'rankdir': 'TB', 'size': '12,12', 'bgcolor': '#f0f8ff'})

    # Define color palette
    colors = {
        'input': '#b3e6ff',
        'embedding': '#ffd1dc',
        'transformation': '#ccffcc',
        'attention': '#ffffb3',
        'normalization': '#ffb3b3',
        'output': '#b3e6ff'
    }

    # Input nodes
    with graph.subgraph(name='cluster_input', node_attr={'shape': 'rectangle', 'style': 'filled'}) as input_subgraph:
        input_subgraph.attr(label='Input', color='#b3e6ff', style='filled')
        input_subgraph.node('sentence_embedding', 'Sentence Embedding\n[0.2, 0.5, ..., 0.1]\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['input'])
        input_subgraph.node('aspect_embedding', 'Aspect Embedding\n[0.1, 0.3, ..., 0.2]\n<i>Shape: (aspect_length, d_model)</i>', fillcolor=colors['input'])

    # Embedding and transformation nodes
    with graph.subgraph(name='cluster_embedding', node_attr={'shape': 'rectangle', 'style': 'filled'}) as embedding_subgraph:
        embedding_subgraph.attr(label='Embedding and Transformation', color='#ffd1dc', style='filled')
        embedding_subgraph.node('concatenation', 'Concatenation\nCombines sentence and aspect embeddings\n<i>Shape: (batch_size, seq_length, 2 * d_model)</i>', fillcolor=colors['embedding'])
        embedding_subgraph.node('query_linear', 'Query Linear\nTransforms concatenated embeddings\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['transformation'])
        embedding_subgraph.node('layer_norm', 'Layer Normalization\nNormalizes the query tensor\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['normalization'])
        embedding_subgraph.node('pos_encoding', 'Positional Encoding\nAdds positional information to the query\n<i>Shape: (seq_length, d_model)</i>', fillcolor=colors['transformation'])

    # Linear transformations for Q, K, and V
    with graph.subgraph(name='cluster_qkv', node_attr={'shape': 'rectangle', 'style': 'filled'}) as qkv_subgraph:
        qkv_subgraph.attr(label='Linear Transformations (Q, K, V)', color='#ccffcc', style='filled')
        qkv_subgraph.node('wq', 'Linear (Q)\nCreates query vectors\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['transformation'])
        qkv_subgraph.node('wk', 'Linear (K)\nCreates key vectors\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['transformation'])
        qkv_subgraph.node('wv', 'Linear (V)\nCreates value vectors\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['transformation'])

    # Reshaping and multi-head attention nodes
    with graph.subgraph(name='cluster_attention', node_attr={'shape': 'rectangle', 'style': 'filled'}) as attention_subgraph:
        attention_subgraph.attr(label='Multi-Head Attention', color='#ffffb3', style='filled')
        attention_subgraph.node('reshape_q', 'Reshape (Q)\n<i>Shape: (batch_size, num_heads, seq_length, depth)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('reshape_k', 'Reshape (K)\n<i>Shape: (batch_size, num_heads, seq_length, depth)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('reshape_v', 'Reshape (V)\n<i>Shape: (batch_size, num_heads, seq_length, depth)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('matmul_qk', 'Matrix Multiplication (Q, K)\nComputes attention scores\n<i>Shape: (batch_size, num_heads, seq_length, seq_length)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('scale', 'Scaling\nScales attention scores\n<i>Shape: (batch_size, num_heads, seq_length, seq_length)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('softmax', 'Softmax\nNormalizes attention scores\n<i>Shape: (batch_size, num_heads, seq_length, seq_length)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('dropout', 'Dropout\nRegularizes attention scores\n<i>Shape: (batch_size, num_heads, seq_length, seq_length)</i>', fillcolor=colors['attention'])
        attention_subgraph.node('matmul_av', 'Matrix Multiplication (Attention, V)\nComputes attention-weighted values\n<i>Shape: (batch_size, num_heads, seq_length, depth)</i>', fillcolor=colors['attention'])

    # Output nodes
    with graph.subgraph(name='cluster_output', node_attr={'shape': 'rectangle', 'style': 'filled'}) as output_subgraph:
        output_subgraph.attr(label='Output', color='#b3e6ff', style='filled')
        output_subgraph.node('reshape_output', 'Reshape Output\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['output'])
        output_subgraph.node('fc_out', 'Linear\nTransforms reshaped output\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['output'])
        output_subgraph.node('residual', 'Residual Connection\nAdds query to the transformed output\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['output'])
        output_subgraph.node('layer_norm_output', 'Layer Normalization\nNormalizes the output\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['normalization'])
        output_subgraph.node('output', 'Output\nContextual Embeddings\n[0.7, 0.2, ..., 0.5]\n<i>Shape: (batch_size, seq_length, d_model)</i>', fillcolor=colors['output'])
        output_subgraph.node('attention_weights', 'Attention Weights\n[[0.1, 0.2, ..., 0.05], ...]\n<i>Shape: (batch_size, num_heads, seq_length, seq_length)</i>', fillcolor=colors['attention'])

    # Edges
    graph.edge('sentence_embedding', 'concatenation', label='sentence_embedding')
    graph.edge('aspect_embedding', 'concatenation', label='aspect_embedding')
    graph.edge('concatenation', 'query_linear', label='concatenated_query')
    graph.edge('query_linear', 'layer_norm', label='query')
    graph.edge('layer_norm', 'pos_encoding', label='normalized_query')
    graph.edge('pos_encoding', 'wq', label='query')
    graph.edge('pos_encoding', 'wk', label='query')
    graph.edge('pos_encoding', 'wv', label='query')
    graph.edge('wq', 'reshape_q', label='Q')
    graph.edge('wk', 'reshape_k', label='K')
    graph.edge('wv', 'reshape_v', label='V')
    graph.edge('reshape_q', 'matmul_qk', label='Q')
    graph.edge('reshape_k', 'matmul_qk', label='K')
    graph.edge('matmul_qk', 'scale', label='attention_scores')
    graph.edge('scale', 'softmax', label='scaled_attention_scores')
    graph.edge('softmax', 'dropout', label='attention_probs')
    graph.edge('dropout', 'matmul_av', label='attention')
    graph.edge('reshape_v', 'matmul_av', label='V')
    graph.edge('matmul_av', 'reshape_output', label='weighted_value')
    graph.edge('reshape_output', 'fc_out', label='reshaped_output')
    graph.edge('fc_out', 'residual', label='output')
    graph.edge('pos_encoding', 'residual', label='query')
    graph.edge('residual', 'layer_norm_output', label='residual_output')
    graph.edge('layer_norm_output', 'output', label='output')
    graph.edge('dropout', 'attention_weights', label='attention')

    # Legend
    with graph.subgraph(name='cluster_legend', node_attr={'shape': 'plaintext'}) as legend_subgraph:
        legend_subgraph.attr(label='Legend', color='black', style='filled', fillcolor='white')
        legend_subgraph.node('legend', '''<
            <table border="0" cellborder="1" cellspacing="0" cellpadding="4">
            <tr><td bgcolor="{input}">Input/Output</td></tr>
            <tr><td bgcolor="{embedding}">Embedding</td></tr>
            <tr><td bgcolor="{transformation}">Transformation</td></tr>
            <tr><td bgcolor="{attention}">Attention</td></tr>
            <tr><td bgcolor="{normalization}">Normalization</td></tr>
            </table>
        >'''.format(**colors))

    # Render and save the diagram
    graph.render('multihead_attention_with_aspect_improved', view=True)

# Call the function to generate the diagram
visualize_multihead_attention_with_aspect()

In [21]:
import graphviz

def visualize_multihead_attention_with_aspect_simplified():
    graph = graphviz.Digraph(comment='MultiheadAttentionWithAspect (Simplified)', graph_attr={'rankdir': 'TB', 'size': '8,8', 'bgcolor': '#f0f8ff'})

    colors = {
        'input': '#b3e6ff',
        'transformation': '#ccffcc',
        'attention': '#ffffb3',
        'output': '#b3e6ff'
    }

    # Input nodes
    graph.node('sentence_embedding', 'Sentence Embedding', shape='rectangle', style='filled', fillcolor=colors['input'])
    graph.node('aspect_embedding', 'Aspect Embedding', shape='rectangle', style='filled', fillcolor=colors['input'])

    # Transformation nodes
    graph.node('concatenation', 'Concatenation', shape='rectangle', style='filled', fillcolor=colors['transformation'])
    graph.node('query_linear', 'Query Linear', shape='rectangle', style='filled', fillcolor=colors['transformation'])
    graph.node('add_position', 'Add Positional Encoding', shape='rectangle', style='filled', fillcolor=colors['transformation'])
    graph.node('qkv_linear', 'Linear (Q, K, V)', shape='rectangle', style='filled', fillcolor=colors['transformation'])

    # Attention nodes
    graph.node('attention', 'Multi-Head Attention', shape='rectangle', style='filled', fillcolor=colors['attention'])

    # Output nodes
    graph.node('output', 'Output', shape='rectangle', style='filled', fillcolor=colors['output'])

    # Edges
    graph.edge('sentence_embedding', 'concatenation', label='sentence_embedding')
    graph.edge('aspect_embedding', 'concatenation', label='aspect_embedding')
    graph.edge('concatenation', 'query_linear', label='concatenated_query')
    graph.edge('query_linear', 'add_position', label='query')
    graph.edge('add_position', 'qkv_linear', label='query_with_position')
    graph.edge('qkv_linear', 'attention', label='Q, K, V')
    graph.edge('attention', 'output', label='attended_output')

    graph.render('multihead_attention_with_aspect_simplified', view=True)

visualize_multihead_attention_with_aspect_simplified()

In [13]:
import graphviz

def visualize_overall_process():
    # Create a new directed graph
    graph = graphviz.Digraph(comment='Overall Process', graph_attr={'rankdir': 'TB'})

    # Input nodes
    graph.node('sentence', 'Raw Sentence\n"The food was delicious, but the service was slow."', shape='box', style='filled', fillcolor='lightblue')
    graph.node('aspect', 'Aspect\n"food", "service"', shape='box', style='filled', fillcolor='lightblue')
    graph.node('polarity', 'Polarity\n"positive", "negative"', shape='box', style='filled', fillcolor='lightblue')

    # RoBERTa embeddings
    graph.node('roberta', 'RoBERTa\nGenerates embeddings', shape='rectangle', style='filled', fillcolor='lightgreen')
    graph.edge('sentence', 'roberta')
    graph.edge('aspect', 'roberta')

    # Attention mechanism
    graph.node('attention', 'Attention Mechanism\nGenerates contextual embeddings', shape='rectangle', style='filled', fillcolor='lightgreen')
    graph.edge('roberta', 'attention')

    # Contextual embeddings
    graph.node('contextual_embeddings', 'Contextual Embeddings\n[0.2, 0.5, ...], [0.1, 0.3, ...]', shape='box', style='filled', fillcolor='lightyellow')
    graph.edge('attention', 'contextual_embeddings')

    # Splitting into windows
    graph.node('window_splitting', 'Window Splitting\nSplits embeddings into windows', shape='rectangle', style='filled', fillcolor='lightgreen')
    graph.edge('contextual_embeddings', 'window_splitting')

    # Window embeddings
    graph.node('window_embeddings', 'Window Embeddings\n[[0.2, 0.5, ...], [0.1, 0.3, ...]], [[0.8, 0.2, ...], [0.4, 0.6, ...]]', shape='box', style='filled', fillcolor='lightyellow')
    graph.edge('window_splitting', 'window_embeddings')

    # Contrastive learning
    graph.node('contrastive_learning', 'Contrastive Learning\nLearns sentiment representations', shape='rectangle', style='filled', fillcolor='lightgreen')
    graph.edge('window_embeddings', 'contrastive_learning')
    graph.edge('polarity', 'contrastive_learning')

    # Output node
    graph.node('output', 'Sentiment Predictions\nPositive: [0.8, 0.2]\nNegative: [0.3, 0.7]', shape='box', style='filled', fillcolor='lightblue')
    graph.edge('contrastive_learning', 'output')

    # Render and save the diagram
    graph.render('overall_process', view=True)

# Call the function to generate the diagram
visualize_overall_process()

In [12]:
import graphviz

def visualize_contrastive_model():
    # Create a new directed graph
    graph = graphviz.Digraph(comment='ContrastiveModel', graph_attr={'rankdir': 'TB', 'size': '12,12'})

    # Input nodes
    graph.node('input_ids', 'Input IDs\nShape: (batch_size, seq_length)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('attention_mask', 'Attention Mask\nShape: (batch_size, seq_length)', shape='box', style='filled', fillcolor='lightblue')

    # BERT layer
    graph.node('bert', 'BERT Model\nbert-base-uncased\nOutput Shape: (batch_size, hidden_size)', shape='box', style='filled', fillcolor='lightgreen')
    graph.edge('input_ids', 'bert', label='input_ids')
    graph.edge('attention_mask', 'bert', label='attention_mask')

    # Dropout and linear layers
    graph.node('dropout1', 'Dropout\nDropout rate: 0.5', shape='box', style='filled', fillcolor='lightpink')
    graph.node('fc1', 'Linear\nInput Shape: (batch_size, hidden_size)\nOutput Shape: (batch_size, hidden_dim)', shape='box', style='filled', fillcolor='lightyellow')
    graph.node('relu1', 'ReLU', shape='box', style='filled', fillcolor='lightblue')
    graph.node('dropout2', 'Dropout\nDropout rate: 0.5', shape='box', style='filled', fillcolor='lightpink')
    graph.node('fc2', 'Linear\nInput Shape: (batch_size, hidden_dim)\nOutput Shape: (batch_size, hidden_dim // 2)', shape='box', style='filled', fillcolor='lightyellow')
    graph.node('relu2', 'ReLU', shape='box', style='filled', fillcolor='lightblue')
    graph.node('dropout3', 'Dropout\nDropout rate: 0.5', shape='box', style='filled', fillcolor='lightpink')

    graph.edge('bert', 'dropout1', label='pooled_output')
    graph.edge('dropout1', 'fc1', label='x')
    graph.edge('fc1', 'relu1', label='x')
    graph.edge('relu1', 'dropout2', label='x')
    graph.edge('dropout2', 'fc2', label='x')
    graph.edge('fc2', 'relu2', label='x')
    graph.edge('relu2', 'dropout3', label='x')

    # Aspect-specific linear layers
    graph.node('aspects_fc', 'ModuleList\nmax_aspects Linear layers\nInput Shape: (batch_size, hidden_dim // 2)\nOutput Shape: (batch_size, n_classes)', shape='box', style='filled', fillcolor='lightgreen')
    graph.edge('dropout3', 'aspects_fc', label='x')

    # Output nodes
    graph.node('output_negative', 'Negative\nShape: (batch_size, 1)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('output_neutral', 'Neutral\nShape: (batch_size, 1)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('output_positive', 'Positive\nShape: (batch_size, 1)', shape='box', style='filled', fillcolor='lightblue')
    graph.node('output_other', 'Other\nShape: (batch_size, 1)', shape='box', style='filled', fillcolor='lightblue')

    graph.edge('aspects_fc', 'output_negative', label='aspect_probabilities[0]')
    graph.edge('aspects_fc', 'output_neutral', label='aspect_probabilities[1]')
    graph.edge('aspects_fc', 'output_positive', label='aspect_probabilities[2]')
    graph.edge('aspects_fc', 'output_other', label='aspect_probabilities[3]')

    # Render and save the diagram
    graph.render('contrastive_model', view=True)

def visualize_contrastive_dataset():
    # Create a new directed graph
    graph = graphviz.Digraph(comment='ContrastiveDataset', graph_attr={'rankdir': 'LR', 'size': '12,8'})

    # Input nodes
    graph.node('positive_pairs', 'Positive Pairs\n(tensor1, tensor2)\nSample:\n([0.1, 0.2, ...], [0.3, 0.4, ...])\n([0.5, 0.6, ...], [0.7, 0.8, ...])', shape='box', style='filled', fillcolor='lightblue')
    graph.node('negative_pairs', 'Negative Pairs\n(tensor1, tensor2)\nSample:\n([0.2, 0.3, ...], [0.9, 0.1, ...])\n([0.4, 0.5, ...], [0.6, 0.7, ...])', shape='box', style='filled', fillcolor='lightblue')
    graph.node('positive_sentiments', 'Positive Sentiments\nSample:\n[2, 2]\n[1, 2]', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('negative_sentiments', 'Negative Sentiments\nSample:\n[0, 2]\n[1, 0]', shape='box', style='filled', fillcolor='lightgreen')

    # ContrastiveDataset node
    graph.node('contrastive_dataset', 'ContrastiveDataset\n__init__(positive_pairs, negative_pairs,\npositive_sentiments, negative_sentiments)\n__len__(), __getitem__(idx)', shape='box', style='filled', fillcolor='lightyellow')

    graph.edge('positive_pairs', 'contrastive_dataset', label='pairs')
    graph.edge('negative_pairs', 'contrastive_dataset', label='pairs')
    graph.edge('positive_sentiments', 'contrastive_dataset', label='sentiments')
    graph.edge('negative_sentiments', 'contrastive_dataset', label='sentiments')

    # Output nodes
    graph.node('pairs', 'Pairs\n(tensor1, tensor2)\nSample:\n([0.1, 0.2, ...], [0.3, 0.4, ...])\n([0.2, 0.3, ...], [0.9, 0.1, ...])', shape='box', style='filled', fillcolor='lightblue')
    graph.node('labels', 'Labels\nSample:\n[1, 0]', shape='box', style='filled', fillcolor='lightpink')
    graph.node('sentiments', 'Sentiments\nSample:\n[2, 2]\n[0, 2]', shape='box', style='filled', fillcolor='lightgreen')

    graph.edge('contrastive_dataset', 'pairs', label='self.pairs')
    graph.edge('contrastive_dataset', 'labels', label='self.labels')
    graph.edge('contrastive_dataset', 'sentiments', label='self.sentiments')

    # Render and save the diagram
    graph.render('contrastive_dataset', view=True)

# Call the functions to generate the diagrams
visualize_contrastive_model()
visualize_contrastive_dataset()

In [1]:
import pandas as pd
train_data = "model/dataframe/train-aug.pkl"
test_data = "model/dataframe/test.pkl"
val_data = "model/dataframe/val.pkl"
train_df = pd.read_pickle(train_data)
val_df = pd.read_pickle(test_data)
test_df = pd.read_pickle(val_data)

In [17]:
counts = train_df['Aspect Term'].unique()

print(len(counts))

train_df=train_df.sort_values('id')
display(train_df.head(5))
# filtered_df = train_df[train_df['id'] == 2355]
# (list(filtered_df['Sentence']))

131


,id,Sentence,Aspect Term,polarity_numeric,Aspect Number
1,4,"though the service might be a little slow, the...",staff,2,1
0,4,"though the service might be a little slow, the...",service,0,0
3,14,after reading other reviews i was expecting po...,staff,2,1
2,14,after reading other reviews i was expecting po...,service,0,0
5,20,"i found that the food variety was great, and t...",staff,2,1


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [20]:
import graphviz

def visualize_hyperparameter_tuning():
    # Create a new directed graph
    graph = graphviz.Digraph(comment='Hyperparameter Tuning', graph_attr={'rankdir': 'TB', 'size': '12,12'})

    # Define nodes
    graph.node('start', 'Start', shape='ellipse', style='filled', fillcolor='lightblue')
    graph.node('param_grid', 'Define Parameter Grid', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('subset_loop', 'Subset Loop', shape='box', style='filled', fillcolor='lightyellow')
    graph.node('train_loop', 'Training Loop', shape='box', style='filled', fillcolor='lightyellow')
    graph.node('train_model', 'Train Model', shape='box', style='filled', fillcolor='lightblue')
    graph.node('validation', 'Validation', shape='box', style='filled', fillcolor='lightpink')
    graph.node('log_metrics', 'Log Metrics', shape='box', style='filled', fillcolor='lightgreen')
    graph.node('select_best', 'Select Best Model', shape='box', style='filled', fillcolor='lightblue')
    graph.node('end', 'End', shape='ellipse', style='filled', fillcolor='lightblue')

    # Define edges
    graph.edge('start', 'param_grid')
    graph.edge('param_grid', 'subset_loop', label='For each parameter combination')
    graph.edge('subset_loop', 'train_loop', label='For each subset')
    graph.edge('train_loop', 'train_model', label='For each epoch')
    graph.edge('train_model', 'validation', label='Evaluate on validation set')
    graph.edge('validation', 'log_metrics', label='Log validation metrics')
    graph.edge('log_metrics', 'train_loop', label='Continue training')
    graph.edge('train_loop', 'subset_loop', label='Next subset')
    graph.edge('subset_loop', 'select_best', label='All subsets completed')
    graph.edge('select_best', 'end')

    # Add loop annotations
    graph.edge('train_loop', 'train_loop', label='Repeat for each epoch', dir='back', style='dashed')
    graph.edge('subset_loop', 'subset_loop', label='Repeat for each subset', dir='back', style='dashed')

    # Render and save the diagram
    graph.render('hyperparameter_tuning', view=True)

# Call the function to generate the diagram
visualize_hyperparameter_tuning()